In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import DeltaTable
from random import *
from itertools import product


# Define paths
storage_account_name = "adcampaigndata"
silver_container = "silver"
gold_container = "gold"

spark.conf.set(
  f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
  dbutils.secrets.get(scope="adls-creds", key="storage-key")
)


spark.sql("CREATE SCHEMA IF NOT EXISTS gold")


silver_base = f"abfss://{silver_container}@{storage_account_name}.dfs.core.windows.net/"
gold_base = f"abfss://{gold_container}@{storage_account_name}.dfs.core.windows.net/"

# Define paths to cleaned Silver data (update paths as per your setup)
silver_youtube_path = silver_base + "Youtube_ads"
silver_meta_path =  silver_base + "Meta_ads"
silver_web_path = silver_base + "Web_analytics"

# Load silver tables
df_youtube = spark.read.format("delta").load(silver_youtube_path)
df_meta = spark.read.format("delta").load(silver_meta_path)
df_web = spark.read.format("delta").load(silver_web_path)


# Upsert function
def upsert_to_delta(df, table_name, keys, zorder_cols):
    if DeltaTable.isDeltaTable(spark, f"/mnt/gold/{table_name}"):
        delta_table = DeltaTable.forPath(spark, f"/mnt/gold/{table_name}")
        update_expr = {col: f"updates.{col}" for col in df.columns}
        delta_table.alias("target").merge(
            df.alias("updates"),
            " AND ".join([f"target.{k} = updates.{k}" for k in keys])
        ).whenMatchedUpdate(set=update_expr) \
         .whenNotMatchedInsertAll() \
         .execute()

        if zorder_cols:
            spark.sql(f"OPTIMIZE gold.{table_name} ZORDER BY ({', '.join(zorder_cols)})")
    else:
        df.write.format("delta").mode("overwrite").save(gold_base + f"/{table_name}")
        table_path = gold_base + f"/{table_name}"
        # Register or refresh the table in Hive metastore
        spark.sql(f"""
            CREATE TABLE IF NOT EXISTS gold.{table_name}
            USING DELTA
            LOCATION '{table_path}'
        """)
        if zorder_cols:
                spark.sql(f"OPTIMIZE gold.{table_name} ZORDER BY ({', '.join(zorder_cols)})")
